In [15]:
import numpy as np
import pandas as pd
import xlrd as xl
from pandas import ExcelWriter
from pandas import ExcelFile
import pickle, re, json, os, datetime, time

import pprint
pp = pprint.PrettyPrinter(indent=4)

In [16]:
## Read the data from the pickle file
all_data = pd.read_excel('../data/all_data.xlsx')
print("Size of corpus: "+str(len(all_data)))

Size of corpus: 8536


In [9]:
import numpy as np
import pandas as pd

df = pd.read_excel('../data/all_data.xlsx')

# Find rows where the label is 'nan'
nan_rows = df[df['Code'].isnull()]

# Display the rows containing 'nan'
print("Rows containing 'nan':")
print(nan_rows)

# Drop rows where the label is 'nan'
df = df.dropna(subset=['Code'])

# Convert the 'Code' column to string
df['Code'] = df['Code'].astype(str)

# Get the number of unique classes
num_classes = df['Code'].nunique()
print("\nNumber of unique classes:", num_classes)

# Get the set of all unique knowledge types in the corpus
knowledge_types = df['Code'].unique()
knowledge_types = np.sort(knowledge_types)  # Sort the unique knowledge types

print("Number of unique knowledge types:", len(knowledge_types))

# Print each label in a new line
for label in knowledge_types:
    print(label)



Rows containing 'nan':
Empty DataFrame
Columns: [Text Content, Code]
Index: []

Number of unique classes: 14
Number of unique knowledge types: 14
Bug Reproduction
Contribution and Commitment
Expected Behaviour
Future Plan
Investigation and Exploration
Motivation
Observed Bug Behaviour
Potential New Issues and Requests
Solution Discussion
Solution Usage
Task Progress
Testing
Usage
Workarounds


In [17]:
## Get the set of all unique knowledge types in the corpus

knowledge_types = list(set(all_data['Code']))
knowledge_types = [str(item) for item in knowledge_types]  # Convert all elements to strings
print("Number of unique knowledge types: " + str(len(knowledge_types)))
knowledge_types.sort()

pp.pprint(knowledge_types)


Number of unique knowledge types: 14
[   'Bug Reproduction',
    'Contribution and Commitment',
    'Expected Behaviour',
    'Future Plan',
    'Investigation and Exploration',
    'Motivation',
    'Observed Bug Behaviour',
    'Potential New Issues and Requests',
    'Solution Discussion',
    'Solution Usage',
    'Task Progress',
    'Testing',
    'Usage',
    'Workarounds']


In [18]:
projects = ['tensorflow','scikit-learn','spaCy', 'keras', 'pandas']
for proj in projects:
    print("Number of sentences from "+proj+"issues: "+str(len(all_data[all_data.Document.str.contains(proj)])))

Number of sentences from tensorflowissues: 479
Number of sentences from scikit-learnissues: 80
Number of sentences from spaCyissues: 52
Number of sentences from kerasissues: 320
Number of sentences from pandasissues: 416


In [19]:
all_data.iloc[0:3]

,Document,Code
0,Node.js (JavaScript) Wrapper API,Expected Behaviour
1,Because JavaScript is Awesome,Motivation
2,As quoted from the offical website http://www....,Motivation


Transformation on the data:
Drop Full Length
Convert begauth which contains values True and False to One Hot Encoding
Convert the time-based feature tpos2 to a numeric field.

In [29]:


# Select only the desired columns
transformed_data = all_data[['Document', 'Code']]



print('Done')


Done


In [30]:
transformed_data.iloc[0:1]

,Document,Code
0,Node.js (JavaScript) Wrapper API,Expected Behaviour


Notice that the field Full Length no longer exists and the field begauth has now been changed to begauth_False and begauth_True.



In [32]:
from sklearn.model_selection import train_test_split

X = transformed_data['Document']
y = transformed_data['Code']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [33]:
X = transformed_data['Document']
y = transformed_data['Code']
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10, shuffle=True)

vectorizer = CountVectorizer()
X_train_tf = vectorizer.fit_transform(X_train)
X_test_tf = vectorizer.transform(X_test)

clf = LogisticRegression()
model = clf.fit(X_train_tf, y_train)
y_pred = clf.predict(X_test_tf)

print(classification_report(y_test, y_pred))

                                   precision    recall  f1-score   support

                 Bug Reproduction       0.66      0.54      0.59       148
      Contribution and Commitment       0.86      0.84      0.85       170
               Expected Behaviour       0.28      0.17      0.21        30
                      Future Plan       0.86      0.86      0.86         7
    Investigation and Exploration       0.64      0.46      0.53       167
                       Motivation       0.51      0.39      0.44        71
           Observed Bug Behaviour       0.72      0.40      0.51        90
Potential New Issues and Requests       0.42      0.21      0.28        52
              Solution Discussion       0.67      0.88      0.76       796
                   Solution Usage       0.00      0.00      0.00         4
                    Task Progress       0.61      0.37      0.46        30
                          Testing       0.75      0.50      0.60         6
                        

c:\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [35]:
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, Conv1D, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Split data into training and testing sets
X = transformed_data['Document']
y = transformed_data['Code']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10, shuffle=True)

# Encode labels as integers
le = LabelEncoder()
le.fit(y_train)
y_train = le.transform(y_train)
y_test = le.transform(y_test)

# Convert target variables to one-hot encoding
num_classes = len(knowledge_types)
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# Tokenize and pad sequences
max_sequence_length = 1000
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)
X_train_pad = pad_sequences(X_train_seq, maxlen=max_sequence_length)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_sequence_length)

# Define the model architecture
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=50, input_length=max_sequence_length))
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(rate=0.3))
model.add(Dense(units=num_classes, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train_pad, y_train, validation_data=(X_test_pad, y_test), epochs=10, batch_size=32)

# Evaluate the model on test data
y_pred = model.predict(X_test_pad)
y_pred = np.argmax(y_pred, axis=1)
y_test_decoded = np.argmax(y_test, axis=1)
accuracy = accuracy_score(y_test_decoded, y_pred)
print('Test accuracy:', accuracy)


Epoch 1/10
214/214 [==============================] - 17s 73ms/step - loss: 1.9660 - accuracy: 0.4606 - val_loss: 1.6723 - val_accuracy: 0.5205
Epoch 2/10
214/214 [==============================] - 15s 72ms/step - loss: 1.5419 - accuracy: 0.5403 - val_loss: 1.4152 - val_accuracy: 0.5861
Epoch 3/10
214/214 [==============================] - 16s 73ms/step - loss: 1.2003 - accuracy: 0.6325 - val_loss: 1.3303 - val_accuracy: 0.6036
Epoch 4/10
214/214 [==============================] - 15s 71ms/step - loss: 0.9258 - accuracy: 0.7119 - val_loss: 1.3180 - val_accuracy: 0.6347
Epoch 5/10
214/214 [==============================] - 15s 71ms/step - loss: 0.7208 - accuracy: 0.7800 - val_loss: 1.4197 - val_accuracy: 0.6563
Epoch 6/10
214/214 [==============================] - 16s 73ms/step - loss: 0.5501 - accuracy: 0.8382 - val_loss: 1.4722 - val_accuracy: 0.6522
Epoch 7/10
214/214 [==============================] - 15s 72ms/step - loss: 0.4112 - accuracy: 0.8871 - val_loss: 1.5984 - val_accuracy:

In [36]:
from sklearn.metrics import classification_report

# Convert one-hot encoded test labels back to integer labels
y_test_int = np.argmax(y_test, axis=1)

# Generate predictions for test data
y_pred = model.predict(X_test_pad)
y_pred_int = np.argmax(y_pred, axis=1)

# Print classification report
print(classification_report(y_test_int, y_pred_int))


54/54 [==============================] - 1s 15ms/step
              precision    recall  f1-score   support

           0       0.62      0.70      0.66       148
           1       0.88      0.85      0.87       170
           2       0.29      0.23      0.26        30
           3       1.00      0.14      0.25         7
           4       0.47      0.49      0.48       167
           5       0.32      0.46      0.38        71
           6       0.56      0.48      0.51        90
           7       0.37      0.27      0.31        52
           8       0.76      0.79      0.78       796
           9       0.00      0.00      0.00         4
          10       0.43      0.33      0.38        30
          11       0.75      0.50      0.60         6
          12       0.56      0.42      0.48        71
          13       0.49      0.44      0.46        66

    accuracy                           0.66      1708
   macro avg       0.54      0.44      0.46      1708
weighted avg       0.66   

c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
